# Study on XNLI (french only)

In [4]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
import os
from os import path

import sys
sys.path.append("./../src")

cache_path = path.join(os.getcwd(), '..', '.cache')
dataset_path = path.join(cache_path, 'dataset')
tmp_path = path.join('.cache', '2023-04-13')
os.makedirs(tmp_path,exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from datasets import load_dataset
import shutil

dataset = load_dataset("xnli", 'fr', cache_dir=dataset_path)

Reusing dataset xnli (/Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
HF_LABEL_ITOS = ['implication', 'neutre', 'contradiction']
LABEL_ITOS = ['neutre', 'implication', 'contradiction']
LABEL_STOI = {label: index for index, label in enumerate(LABEL_ITOS)}

In [7]:
df_train = dataset['train'].to_pandas()

# Convert numeric labels into string label to avoid confusion
df_train = df_train.replace({'label': {index: label for index, label in enumerate(HF_LABEL_ITOS)}})
df_train = df_train.rename(columns={'premise': 'premise.text', 'hypothesis': 'hypothesis.text'})
df_train['label'] = df_train['label'].astype('category')

df_train

,premise.text,hypothesis.text,label
0,L' écrémage conceptuel de la crème a deux dime...,Le produit et la géographie sont ce qui fait t...,neutre
1,Tu sais pendant la saison et je suppose qu' à ...,Vous perdez les choses au niveau suivant si le...,implication
2,Un de nos numéros vous fera suivre vos instruc...,Un membre de mon équipe exécutera vos ordres a...,implication
3,Qu' est-ce que tu en sais ? Tout ceci est à no...,Cette information leur appartient .,implication
4,Ouais je te dis ce que si tu vas prix certaine...,Les chaussures de tennis ont une gamme de prix .,neutre
...,...,...,...
392697,"De toute évidence , la Californie peut - et do...",La Californie ne peut pas faire mieux .,contradiction
392698,Elle était autrefois considérée comme la plus ...,Beaucoup de bâtiments originaux avaient été re...,neutre
392699,Les péniches sont une tradition magnifiquement...,La tradition des péniches est née alors que le...,implication
392700,Les nécrologie ont rappelé avec tendresse ses ...,Les nécrologie étaient belles et écrites en na...,neutre


## Pretokenize the text

In [3]:
import spacy
import pandas as pd

# Load spacy tokenizer
sm = spacy.load('fr_core_news_sm')

In [ ]:
%%time

## WARNING : THIS GONNA TAKE A LONG TIME EXECUTION

for side in ['premise', 'hypothesis']:
    docs = sm.pipe(df_train[side+'.text'])
    tokens = [[token.lemma_.lower() for token in sentence] for sentence in docs]
    df_train[side+'.tokens'] = pd.Series(tokens)

In [6]:
df_train = pd.read_json(path.join(dataset_path, 'xnli','fr', 'train.json'))

`json`

* Saving time : 2.14 s
* Loading time : 5.64 s

In [6]:
json_path = path.join(dataset_path, 'xnli','fr', 'train.json')

In [8]:
%%time
## SAVE TIME
df_train.to_json(json_path)

CPU times: user 1.66 s, sys: 425 ms, total: 2.08 s
Wall time: 2.14 s


In [7]:
%%time
## LOAD TIME
df_train = pd.read_json(json_path)

CPU times: user 4.97 s, sys: 674 ms, total: 5.64 s
Wall time: 5.64 s


Check UTF-8 encoding correctly loaded in french

In [10]:
df_train

,premise.text,hypothesis.text,label,premise.tokens,hypothesis.tokens
0,L' écrémage conceptuel de la crème a deux dime...,Le produit et la géographie sont ce qui fait t...,neutre,"[l, ', écrémage, conceptuel, de, le, crème, av...","[le, produit, et, le, géographie, être, ce, qu..."
1,Tu sais pendant la saison et je suppose qu' à ...,Vous perdez les choses au niveau suivant si le...,implication,"[taire, savoir, pendant, le, saison, et, je, s...","[vous, perdre, le, chose, au, niveau, suivre, ..."
2,Un de nos numéros vous fera suivre vos instruc...,Un membre de mon équipe exécutera vos ordres a...,implication,"[un, de, notre, numéro, vous, faire, suivre, v...","[un, membre, de, mon, équipe, exécuter, votre,..."
3,Qu' est-ce que tu en sais ? Tout ceci est à no...,Cette information leur appartient .,implication,"[que, ', être, -ce, que, taire, en, sai, ?, to...","[ce, information, leur, appartient, .]"
4,Ouais je te dis ce que si tu vas prix certaine...,Les chaussures de tennis ont une gamme de prix .,neutre,"[ouai, je, te, dis, ce, que, si, taire, vas, p...","[le, chaussure, de, tennis, avoir, un, gamme, ..."
...,...,...,...,...,...
392697,"De toute évidence , la Californie peut - et do...",La Californie ne peut pas faire mieux .,contradiction,"[de, tout, évidence, ,, le, californie, pouvoi...","[le, californie, ne, pouvoir, pas, faire, mieu..."
392698,Elle était autrefois considérée comme la plus ...,Beaucoup de bâtiments originaux avaient été re...,neutre,"[lui, être, autrefois, considérer, comme, le, ...","[beaucoup, de, bâtiment, original, avoir, être..."
392699,Les péniches sont une tradition magnifiquement...,La tradition des péniches est née alors que le...,implication,"[le, péniche, être, un, tradition, magnifiquem...","[le, tradition, de, péniche, être, naître, alo..."
392700,Les nécrologie ont rappelé avec tendresse ses ...,Les nécrologie étaient belles et écrites en na...,neutre,"[le, nécrologie, avoir, rappeler, avec, tendre...","[le, nécrologie, être, bel, et, écrite, en, na..."


`hdf-5`

* Saving time : 6.57 s
* Loading time : 7.14 s

In [ ]:
hdf_path = path.join(dataset_path, 'xnli','fr', 'data.h5')

In [ ]:
%%time
## SAVE TIME
df_train.to_hdf(hdf_path, key='train', mode='w')

<timed eval>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['premise.text', 'hypothesis.text', 'label', 'premise.tokens',
       'hypothesis.tokens'],
      dtype='object')]



CPU times: user 5.08 s, sys: 1.1 s, total: 6.18 s
Wall time: 6.57 s


In [ ]:
%%time
## LOAD TIME
df_train = pd.read_hdf(hdf_path, key='train', mode='r')

CPU times: user 5.42 s, sys: 1.32 s, total: 6.74 s
Wall time: 7.14 s
